## Project Description

MLB Advanced Media, as stated in a job description for which I was intrigued by, was looking to develop insights into predictability of a hit based on data acquired through their Statcast tool. Statcast is a high-speed, high-accuracy device that tracks ball and player movements. 

The findings of this task would be for use by analysts and commentators during game broadcasts. The problem statement for the specfic prediction I undertook is:

Based on the ballistics of the pitch and the ball hit into play, what is the likelihood it results in a hit.

## Notebook Description

8\. Add **deskewing** method and check score of models again

- Define function to train and test a model using specified predictors and targets
- Deskew data using `box_cox()`
- Apply `MinMaxScaler()` to data
- Train model on deskewed and normalized data
- Models used:
    - K Nearest Neighbors
    - Logistic Regression
    - Decision Tree Classifier
    - Random Forest Classifier
- Results:
    - With addition of `box_cox` skew-normalization, not much changes from findings on `MinMaxScaler` alone.

|   Model Name   |   Test Score  |   Train Score   |
| -----------|:---------------:|--------------:|
| K Nearest Neighbors | 0.6621 | 0.7784 |
| Logistic Regression | 0.7516 | 0.7465 |
| Decision Tree Classifier | 0.7571 | 1.0 |
| Random Forest Classifier | 0.7972 | 0.9875 |

### Initialize packages and read in pickled data

In [1]:
# ! pip install scrapy
# ! pip install psycopg2
# ! pip install sqlalchemy
# ! pip install missingno --quiet
# ! pip install scipy

In [2]:
% run __init__.py

In [3]:
cd ..

/home/jovyan


In [4]:
df_model = pd.read_pickle('data/df_model.p')

In [5]:
df_model.shape

(127052, 88)

### Set up df to `run_benchmark` function on standardized data and compare against un-standardized

In [6]:
df_model.drop('player_id', axis=1, inplace=True)

In [7]:
target = df_model['hit_flag']
predictors = df_model.drop('hit_flag', axis=1)

In [8]:
def run_benchmark(model, model_name, dataframe, target_col):
    target = dataframe[target_col]
    tmp_df = dataframe.drop(target_col, axis=1)
    X_train, X_test, y_train, y_test = train_test_split(tmp_df, target, stratify=target)
    model.fit(X_train, y_train)
    return {'train_score' : model.score(X_train, y_train), 
            'test_score' : model.score(X_test, y_test),
            'model_name' : model_name }

# credit to Joshua Cook

BoxCox requires all positive values, so I'll start this workflow by using a `MinMaxScaler` on my data

### `MinMaxScaler`

In [9]:
df_model_proc_all = predictors.copy()

In [10]:
min_max = MinMaxScaler(feature_range=(1E-10,1))

In [11]:
df_model_mm = pd.DataFrame(min_max.fit_transform(df_model_proc_all), 
                           index=df_model_proc_all.index, 
                           columns=df_model_proc_all.columns)

### Skew-Normalize Features

#### `box_cox`

In [12]:
def box_cox(predictors):
    '''Input dataframe to deskew it'''
    df_model_bc = pd.DataFrame()
    for col in predictors.columns:
        box_cox, lmbda = boxcox(predictors[col])
        df_model_bc[col] = pd.Series(box_cox)
    
    df_model_bc.set_index(predictors.index, inplace=True)
    
    return df_model_bc

In [13]:
df_model_skewnorm = box_cox(df_model_mm)

/opt/conda/lib/python3.6/site-packages/scipy/stats/morestats.py:901: RuntimeWarning: overflow encountered in square
  llf -= N / 2.0 * np.log(np.sum((y - y_mean)**2. / N, axis=0))


In [14]:
df_model_skewnorm.head(3)

,mph,ev_mph,dist,spin_rate,launch_angle,zone_1.0,zone_11.0,zone_12.0,zone_13.0,zone_14.0,...,full_pitch_Knuckle-curve,full_pitch_Knuckleball,full_pitch_Pitch out,full_pitch_Screwball,full_pitch_Slider,full_pitch_Two-Seam Fastball,full_pitch_Unidentified,pitch_rollup_fastball,pitch_rollup_offspeed,pitch_rollup_other
unique_id,,,,,,,,,,,,,,,,,,,,,
434378-8,-0.180440,-0.090954,-0.468592,-0.302719,-0.225151,-1.128437e+08,-1.165759e+09,-1.445784e+10,-1.838004e+08,-1.368555e+07,...,-1.275003e+21,-3.990265e+98,-1.341265e+154,-1.340852e+154,0.000000,-1496.250076,-5.213913e+128,-10.838925,0.000000,-6.834172e+115
434378-14,-0.116816,-0.191493,-0.571264,-0.328504,-0.204774,0.000000e+00,-1.165759e+09,-1.445784e+10,-1.838004e+08,-1.368555e+07,...,-1.275003e+21,-3.990265e+98,-1.341265e+154,-1.340852e+154,-2602.974371,-1496.250076,-5.213913e+128,0.000000,-65.018139,-6.834172e+115
434378-16,-0.161768,-0.261724,-0.721032,-0.320188,-0.241653,-1.128437e+08,-1.165759e+09,-1.445784e+10,-1.838004e+08,-1.368555e+07,...,-1.275003e+21,-3.990265e+98,-1.341265e+154,-1.340852e+154,0.000000,-1496.250076,-5.213913e+128,-10.838925,0.000000,-6.834172e+115


### Standardize Features

**`MinMaxScaler()` on All Features**

In [15]:
standardized = (MinMaxScaler().fit_transform(df_model_skewnorm))
df_standardized = pd.DataFrame(standardized, columns=df_model_skewnorm.columns, index=df_model_skewnorm.index)

In [16]:
df_standardized.shape

(127052, 86)

In [17]:
target.shape

(127052,)

In [18]:
df_model_st = pd.concat([df_standardized, target], axis=1)
# df_model_st = pd.concat([df_model_skewnorm, target], axis=1)

### Run Models on Skew-Normalized and Standardized Data

### K Neighbors Classifier

K Neighbors takes a long time to run. Specifically, the model scoring.

In [25]:
knn_output = run_benchmark(KNeighborsClassifier(n_jobs=7),
                           'kneighbors',
                           df_model_st, 
                           'hit_flag')

In [26]:
knn_output

{'model_name': 'kneighbors',
 'test_score': 0.66212259547271979,
 'train_score': 0.77841093935291583}

### Logistic Regression

In [19]:
log_reg_output = run_benchmark(LogisticRegression(), 
                               'logistic regression',
                               df_model_st, 
                               'hit_flag')

In [20]:
log_reg_output

{'model_name': 'logistic regression',
 'test_score': 0.75156628781916068,
 'train_score': 0.74645551952481404}

### Decision Tree

In [21]:
dtree_output = run_benchmark(DecisionTreeClassifier(), 
                             'decision tree',
                             df_model_st, 
                             'hit_flag')

In [22]:
dtree_output

{'model_name': 'decision tree',
 'test_score': 0.75707584296193686,
 'train_score': 1.0}

### Random Forest Classifier

In [23]:
rand_forest_output = run_benchmark(RandomForestClassifier(), 
                                  'random forest', 
                                  df_model_st, 
                                  'hit_flag')

In [24]:
rand_forest_output

{'model_name': 'random forest',
 'test_score': 0.79724837074583632,
 'train_score': 0.98754315818195171}

### Show benchmark models side-by-side

In [28]:
output = [
    knn_output,
    log_reg_output,
    dtree_output,
    rand_forest_output
]

pd.DataFrame(output)

,model_name,test_score,train_score
0,kneighbors,0.662123,0.778411
1,logistic regression,0.751566,0.746456
2,decision tree,0.757076,1.000000
3,random forest,0.797248,0.987543


With addition of `box_cox` skew-normalization, not much changes from findings on `MinMaxScaler` alone.